In [338]:
%run ./PokemonLib.ipynb

In [354]:
def gatherDataForArena(locations, game, generation, starters, leaderData, FISHING):
    areas = produceAreaForLocations(locations)
    addEncounterablePokemonsToAreas(areas, FISHING)
    
    catchablePokemons = producePokemonForAreas(areas, generation)
    appendPokemonFromStarters(starters, catchablePokemons, generation)
    appendPokemonFromEvolutions(catchablePokemons, game, generation)
    
    
    arenaLeader = produceArenaLeader(leaderData[0], generation, leaderData[1])

    return {'areas': areas, 'catchablePokemons': catchablePokemons, 'arenaLeader': arenaLeader}

In [340]:
def findEvolutionsRecursive(evolvesTo, originalName, game):
    entwicklungen = []
    for entwicklung in evolvesTo:
        entwicklungsName = entwicklung['species']['name']
        entwicklungPokeJson = requestPokemon(entwicklungsName)
        pokeIsInWantedGame = False
        for gameIndice in entwicklungPokeJson['game_indices']:
            if gameIndice['version']['name'] == game:
                pokeIsInWantedGame = True
        if(entwicklungsName != originalName):
          #  print(originalName)
          #  print('entwickelt sich zu')
          #  print(entwicklungsName)
            evoDetails = entwicklung['evolution_details'][0]
            trigger = evoDetails['trigger']['name']
            if trigger == 'level-up':
                levelUpCondition = evoDetails['min_level']
            elif trigger == 'use-item':
                levelUpCondition = evoDetails['item']['name']
            elif trigger == 'trade':
                levelUpCondition = 'trade'
            else:
                levelUpCondition = None
           # print(levelUpCondition)
           # print('----')
            entwicklungen.append((entwicklungsName, levelUpCondition))
            #print(entwicklungen)
        if(entwicklung['evolves_to']!= None):
            entwicklungInDepth = findEvolutionsRecursive(entwicklung['evolves_to'], originalName, game)
            if(len(entwicklungInDepth)>0):
                for entw in entwicklungInDepth:
                    entwicklungen.append(entw)
    return entwicklungen

In [341]:
def fetchPokemonEvolutionPossibilities(catchablePokemons, game):
    allEvolves = []
    for poke in catchablePokemons:
        originalName = catchablePokemons[poke].name
        originalEncounterCondition = catchablePokemons[poke].encounterOrEvolutionTrigger
        pokeJson = fetchPokemon(originalName)
        speciesUrl = pokeJson['species']['url']
        speciesJson = requestFullUrl(speciesUrl)
        evolutionChainUrl = speciesJson['evolution_chain']['url']
        evolutionChainJson = requestFullUrl(evolutionChainUrl)
        chain = evolutionChainJson['chain']
        tempEvolvesTo = chain['evolves_to']
        evolves = findEvolutionsRecursive(tempEvolvesTo, originalName, game)
        for evolve in evolves:
            evolve = (evolve[0], evolve[1], originalEncounterCondition)
            allEvolves.append(evolve)
    return allEvolves

In [342]:
def appendPokemonFromEvolutions(catchablePokemons, game, generation):
    tempEvos = fetchPokemonEvolutionPossibilities(catchablePokemons, game)
    
    for evolution in tempEvos:
        evoName = evolution[0]
        if not evoName in catchablePokemons:
            catchablePokemons[evoName] = producePokemonFromEvolution(evolution, generation)

In [343]:
def appendPokemonFromStarters(starters, pokemonSet, generation):
     for name in starters:
        if not name in pokemonSet:
            pokemonSet[name] = producePokemon(name,generation, 'starter')

In [344]:
def produceAreaForLocation(location, steps):
    areaTemp = fetchAreaToLocation(location)
    area = Area(areaTemp['name'], 0.2, areaTemp, steps )
    return area

In [345]:
def produceAreaForLocations(locations):
    areas = []
    for location in locations:
        locationName = location['name']
        steps = location['steps']
        areas.append(produceAreaForLocation(locationName, steps))
    return areas

In [346]:
def addEncounterablePokemonsToArea(area, FISHING):
     area.encounterablePokemons = returnPokemonAndChanceAndEncounterTypeFromArea(area.json ,[GAME], FISHING)

In [347]:
def addEncounterablePokemonsToAreas(areas, FISHING):
    for area in areas:
        addEncounterablePokemonsToArea(area, FISHING)
        area.calculateAverageEncounters()

In [348]:
def producePokemonForArea(area, pokemonSet, generation):
    for pokemon in area.encounterablePokemons:
        name = pokemon[0]
        encounterType = pokemon[2]
        if not name in pokemonSet:
            pokemonSet[name] = producePokemon(name, generation, encounterType )

In [349]:
def producePokemonForAreas(areas, generation):
    pokemonSet = {}
    for area in areas:
        producePokemonForArea(area, pokemonSet, generation)
    return pokemonSet

In [350]:
def producePokemon(name, generation, encounterType):
    moves = fetchPokemonMoves(name, generation)
    pokemonInstance = Pokemon(name, moves, encounterType)
    return pokemonInstance

In [351]:
def producePokemonFromEvolution(evolution, generation):
    evoName = evolution[0]
    evoTrigger = evolution[1]
    evoOrigin = evolution[2]
    moves = fetchPokemonMoves(evoName, generation)
    pokemonInstance = Pokemon(evoName, moves, evoTrigger)
    pokemonInstance.setOriginIfEvolution(evoOrigin)
    return pokemonInstance

In [352]:
def producePokemonWithCustomMoves(name, generation, customMoves):
    moves = []
    for move in customMoves:
        moves.append(produceMove(move))
    pokemonInstance = Pokemon(name, moves, 'encounter')
    return pokemonInstance

In [353]:
#pokemonTeam = [{name: 'pikachu', level: 12, moves:['tackle']}]
#produceArenaLeader('brock', 'red-blue', [{'name': 'pikachu', 'level': 12, 'moves':['tackle']}])
def produceArenaLeader(name, generation, pokemonTeam):
    pokemon = []
    for poke in pokemonTeam:
        producedPoke = producePokemonWithCustomMoves(poke['name'],generation,poke['moves'])
        pokemon.append((producedPoke, poke['level']))
    
    leader = ArenaLeader(name, pokemon)
    return leader